In [7]:


import pandas as pd
import numpy as np
import plotnine as p9
import matplotlib.pyplot as plt
import os


date_time = "2023-07-30_19:34"
date_time = "2023-07-31_19:32"

# date_time = "2023-09-08_09:59"
# date_time = "2023-09-08_10:26"
# date_time = "2023-09-08_12:09"
# date_time = "2023-09-08_12:23"
# date_time = "2023-09-08_15:27"
# date_time = "2023-09-10_15:41"
# date_time = "2023-09-08_16:06"

# AROON_Oscillator CrossOver
date_time = "2023-09-10_15:41"
date_time = "2023-09-10_18:40"

# MACD CrossOver
date_time = "2023-09-10_19:07"


root_results_dir = "/projects/genomic-ml/da2343/ml_project_2/hpc/accuracy_tune/orders"
df = pd.read_csv(f"{root_results_dir}/{date_time}_results.csv")
# SELL Signal
# df = df[(df["RSI"] > 60) & (df["ADX"] > 40) & (df["AROON_Oscillator"] == -100)]
# df = df[(df["RSI"] > 60) & (df["ADX"] > 40) & (df["CCI"] > 60)]
# df = df[(df["AROON_Oscillator"] == 100) | (df["AROON_Oscillator"] == -100)]

# df = df[df["year"] == 2021]
# arrange year by ascending order
df = df.sort_values(by=["index"]).reset_index(drop=True)

# BUY Signal
# df = df[(df["VOLUME_RSI"] < 70) & (df["ADX"] < 30) & (df["position"] != 0)]

In [8]:
df

,index,ask_price,take_profit_price,stop_loss_price,position,SMA_200,MACD,MACD_Signal,MACD_Hist,RSI,...,OBV,CCI,PSAR,AD,ADOSC,VOLUME_RSI,MFI,label,year,dataset_name
0,23810,1.33475,1.34225,1.32975,1,1.318826,-0.000433,-0.000460,0.000027,56.064519,...,-147942.0,120.828118,1.325308,-1.632164e+05,1726.451332,59.340383,65.891409,0.0,2011,EURUSD_H1
1,23823,1.33420,1.32670,1.33920,0,1.320301,0.000695,0.000767,-0.000072,50.498029,...,-141036.0,-78.149406,1.339450,-1.706056e+05,-1255.302779,53.306053,58.939885,0.0,2011,EURUSD_H1
2,23825,1.33639,1.34389,1.33139,1,1.320574,0.000782,0.000762,0.000020,55.841795,...,-136632.0,42.133873,1.339188,-1.683606e+05,61.589698,49.266933,54.957736,0.0,2011,EURUSD_H1
3,23826,1.33531,1.32781,1.34031,0,1.320707,0.000739,0.000757,-0.000019,52.669074,...,-138986.0,-30.663531,1.339061,-1.684446e+05,216.126013,49.582969,58.497843,1.0,2011,EURUSD_H1
4,23831,1.34002,1.34752,1.33502,1,1.321347,0.000857,0.000677,0.000180,64.040274,...,-139813.0,319.463699,1.332330,-1.608563e+05,2933.786933,66.314733,53.409865,0.0,2011,EURUSD_H1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5427,97852,1.05261,1.06011,1.04761,1,1.045450,0.000500,0.000440,0.000060,60.842322,...,-147344.0,173.359198,1.049390,-3.943356e+06,1987.818037,53.278262,45.785974,1.0,2022,EURUSD_H1
5428,97854,1.04948,1.04198,1.05448,0,1.045597,0.000435,0.000466,-0.000032,46.223492,...,-146105.0,-40.553394,1.053080,-3.941327e+06,3365.122049,51.356737,46.367054,0.0,2022,EURUSD_H1
5429,97858,1.05204,1.05954,1.04704,1,1.045913,0.000491,0.000457,0.000034,55.670497,...,-132439.0,114.744266,1.048950,-3.933935e+06,4598.091153,59.762323,65.303930,1.0,2022,EURUSD_H1
5430,97876,1.05680,1.04930,1.06180,0,1.047387,0.001546,0.001555,-0.000009,60.405667,...,-104576.0,53.826936,1.055876,-3.920359e+06,-765.642993,55.836260,59.670062,0.0,2022,EURUSD_H1


In [3]:
# save df
df.to_csv(f"trades_EURUSD_H1_2011_2022.csv", index=False)

In [ ]:
Xy_df = df[['position', 'RSI', 'ADX', 'WILLR', 'AROON_Oscillator','label']]
X = Xy_df.iloc[:, :-1]
y = Xy_df.iloc[:, -1]

In [46]:
from sktime.forecasting.model_selection import SlidingWindowSplitter

splitter = SlidingWindowSplitter(window_length=20, 
                                 fh=1, 
                                 step_length=1)

accuracy_list = []

for j, (train_idx, test_idx) in enumerate(splitter.split(y)):
    X_train, y_train = X.iloc[train_idx, :].to_numpy(), y.iloc[train_idx].to_numpy()
    X_test, y_test = X.iloc[test_idx, :].to_numpy(), y.iloc[test_idx].to_numpy()
    # classifier = TimeSeriesForestClassifier()
    # classifier.fit(X_train, y_train)
    # y_pred = classifier.predict(X_test)
    y_pred = y_train[-1]
    print(f"y_pred: {y_pred}, y_test: {y_test}")
    # accuracy_list.append(accuracy_score(y_test, y_pred))
    # print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
mean_accuracy = np.mean(accuracy_list)
print(f"Mean Accuracy: {mean_accuracy}")


y_pred: 0.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0, y_test: [1.]
y_pred: 1.0, y_test: [0.]
y_pred: 0.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0, y_test: [0.]
y_pred: 0.0, y_test: [0.]
y_pred: 0.0, y_test: [0.]
y_pred: 0.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0, y_test: [0.]
y_pred: 0.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0, y_test: [0.]
y_pred: 0.0, y_test: [1.]
y_pred: 1.0,

In [ ]:
mean_accuracy = np.mean(accuracy_list)
print(f"Mean Accuracy: {mean_accuracy}")


In [ ]:
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.datasets import load_arrow_head
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# X, y = load_arrow_head()
# X_train, X_test, y_train, y_test = train_test_split(X, y)
classifier = TimeSeriesForestClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
# Make a new accuracy df with columns: year, position, accuracy from the above df using columns 'label', 'position', 'year'

import warnings
warnings.filterwarnings('ignore')

new_df = pd.DataFrame(columns=["year", "position", "accuracy"])
# year starts from 2011 to 2022
# position can be 1 or 0
# accuracy is the accuracy of the position for that year
# accuracy = (number of 1's in the label for that position and year) / (total number of labels for that position and year)
for year in range(2011, 2023):
    for position in [0, 1]:
        try:
            accuracy = df[(df["year"] == year) & (df["position"] == position)]["label"].value_counts()[position] / df[(df["year"] == year) & (df["position"] == position)]["label"].value_counts().sum()
        except:
            accuracy = 0
        new_df = new_df.append({"year": year, "position": position, "accuracy": accuracy}, ignore_index=True)
        
new_df

In [ ]:

# Import seaborn library
import seaborn as sns

# Create a dataframe from the data
new_df = pd.DataFrame(columns=["year", "position", "accuracy"])
# year starts from 2011 to 2022
# position can be 1 or 0
# accuracy is the accuracy of the position for that year
# accuracy = (number of 1's in the label for that position and year) / (total number of labels for that position and year)
for year in range(2011, 2023):
    for position in [0, 1]:
        try:
            accuracy = df[(df["year"] == year) & (df["position"] == position)]["label"].value_counts()[position] / df[(df["year"] == year) & (df["position"] == position)]["label"].value_counts().sum()
        except:
            accuracy = 0
        new_df = new_df.append({"year": year, "position": position, "accuracy": accuracy}, ignore_index=True)


# Plot a stacked bar graph with accuracy on y-axis and year on x-axis
fig, ax = plt.subplots(figsize=(11.7, 8.27)) # A4 paper dimensions
sns.barplot(x="year", 
            y="accuracy", 
            hue="position", 
            data=new_df, 
            ax = ax,
            )
# change the size of the plot

# Add labels and title
plt.xlabel("Year")
plt.ylabel("Accuracy")
plt.title("MACD and AROON accuracy")

# Show the plot
plt.show()

# save the plot
fig.savefig(f"MACD_and AROON_accuracy_by_year_and_position.png", dpi=300, bbox_inches='tight')

In [ ]:
folders_path = "/projects/genomic-ml/da2343/ml_project_2/data"
dataset_list = ["EURUSD_H1", "USDJPY_H1", "GBPUSD_H1",
                "AUDUSD_H1", "USDCAD_H1", "USDCHF_H1", 
                "EURJPY_H1", "EURGBP_H1"]


for dataset_name in dataset_list:
    raw_df = df[df['dataset_name'] == dataset_name]
    # remove the _H1 from the dataset_name
    symbol = dataset_name[:-3]
    
    raw_df_dir = os.path.join(folders_path, symbol, f"{symbol}_H1_2011_2015_TRADES.csv")
    raw_df.to_csv(raw_df_dir, index=False)
    
    wrangled_df_dir = os.path.join(folders_path, symbol, f"{symbol}_H1_2011_2015_TRADES_REAL.csv")
    # wrangled_df = raw_df[['position', 'RSI', 'ADX', 'WILLR', 'AROON_Oscillator', 'label']]
    wrangled_df = raw_df[['position', 'RSI', 'ADX','label']]
    wrangled_df.reset_index(drop=True, inplace=True)
    wrangled_df = wrangled_df.assign(index=wrangled_df.index)
    wrangled_df.to_csv(wrangled_df_dir, index=False)
    
# print(wrangled_df)
wrangled_df


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PowerTransformer

trade_df_test = pd.read_csv('/projects/genomic-ml/da2343/ml_project_2/data/EURUSD/EURUSD_H1_2016_2022_TRADES.csv')
# trade_df_test = pd.read_csv('/projects/genomic-ml/da2343/ml_project_2/hpc/accuracy_tune/orders/2023-07-30_19:34_results.csv')
trade_df_test = trade_df_test[['position', 'RSI', 'ADX', 'label', 'year']]
# trade_df_test = trade_df_test[trade_df_test['year'] == 2016]
trade_df_test.drop('year', axis=1, inplace=True)


# trade_df = pd.read_csv('/projects/genomic-ml/da2343/ml_project_2/data/EURUSD/EURUSD_H1_2011_2015_TRADES.csv')
# trade_df = trade_df[[ 'MACD', 'MACD_Signal', 'MACD_Hist', 'RSI', 'ADX', 'AROON_Oscillator', 'WILLR', 'label']]
# trade_df = trade_df[['position', 'MACD', 'MACD_Signal', 'MACD_Hist', 'RSI', 'ADX', 'AROON_Oscillator', 'WILLR', 'label']]
# trade_df = trade_df[['position', 'RSI', 'ADX', 'WILLR', 'label']]
# trade_df_test = trade_df_test[['position', 'RSI', 'ADX', 'label']]


# reset index
# trade_df.reset_index(drop=True, inplace=True)

# add an index column to trade_df
# trade_df['index'] = trade_df.index

# save trade_df to csv
# trade_df.to_csv('/projects/genomic-ml/da2343/ml_project_2/data/EURUSD/EURUSD_H1_2011_2015_TRADES_binary.csv', index=False)

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Load data
# df = pd.read_csv("df.csv")

# Create a scatter plot of ADX vs RSI with label and position as the hue
# x_name = "AROON_Oscillator"
x_name = "ADX"
y_name = "RSI"
sns.scatterplot(data=df, 
                x=x_name, 
                y=y_name, 
                hue="label", 
                style="position")
plt.xlabel(x_name)
plt.ylabel(y_name)
plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PowerTransformer

# trade_df = pd.read_csv('/projects/genomic-ml/da2343/ml_project_2/data/EURUSD/EURUSD_H1_2011_2015_TRADES_synthetic.csv')
trade_df = pd.read_csv('/projects/genomic-ml/da2343/ml_project_2/data/EURUSD/EURUSD_H1_2011_2015_TRADES_REAL.csv')
# drop index column
trade_df = trade_df.drop(columns=['index'])

In [ ]:
trade_df

In [ ]:
trade_df_test

In [ ]:

import pandas as pd
import numpy as np
import plotnine as p9
from sklearn.model_selection import *
from sklearn.feature_selection import *
from sklearn.preprocessing import *
from sklearn.pipeline import *

from sklearn.linear_model import *
from sklearn.neighbors import *
from sklearn.model_selection import *

from sklearn.model_selection import RandomizedSearchCV

from sklearn.dummy import DummyClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import *
import matplotlib.pyplot as plt
import matplotlib
import plotnine as p9

np.set_printoptions(threshold=np.inf)


X_train, y_train = trade_df.iloc[:, :-1].to_numpy(), trade_df.iloc[:, -1]
X_test, y_test = trade_df_test.iloc[:, :-1].to_numpy(), trade_df_test.iloc[:, -1]

# Create and fit random forest model
model = RandomForestClassifier()

# Define parameter grid for GridSearchCV
param_grid = {"n_estimators": [10, 50, 100, 200], "max_depth": [None, 5, 10, 15], "criterion": ["gini", "entropy"]}

# Create GridSearchCV object with precision score as the scoring function
grid_search = RandomizedSearchCV(model, param_distributions=param_grid, scoring="average_precision", cv=5, n_iter = 100)

# Fit grid search on training data
# grid_search.fit(X_train, y_train)

# model = RandomForestClassifier()
grid_search.fit(X_train, y_train)
# y_pred = grid_search.predict(X_test)

y_pred = grid_search.best_estimator_.predict(X_test)


tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
# tn_accuracy = tn / (tn + fp)
# win_rate = tp / (tp + fp)
            
test_acc_dict = {
    "test_accuracy_percent": accuracy_score(y_test, y_pred) * 100,
    # "tn_accuracy": tn_accuracy * 100,
    # "tp_accuracy": tp_accuracy * 100,
    "precision_score": precision_score(y_test, y_pred),
    # "algorithm": algorithm
}

print(test_acc_dict)

In [ ]:
y_pred

In [ ]:
# {'test_accuracy_percent': 52.87356321839081, 'tn_accuracy': 71.15384615384616, 'tp_accuracy': 25.71428571428571, 'precision_score': 0.375, 'f1_score': 0.30508474576271183}

In [ ]:

# FORECASTING + MACD CROSSOVER
# {'test_accuracy_percent': 55.66218809980806, 'tn_accuracy': 69.64285714285714} for all the years

# {'test_accuracy_percent': 52.87356321839081, 'tn_accuracy': 71.15384615384616} for 2022
# {'test_accuracy_percent': 39.34426229508197, 'tn_accuracy': 65.21739130434783} for 2021
# {'test_accuracy_percent': 58.46153846153847, 'tn_accuracy': 76.19047619047619} for 2020
# {'test_accuracy_percent': 56.25, 'tn_accuracy': 58.92857142857143} for 2019
# {'test_accuracy_percent': 60.49382716049383, 'tn_accuracy': 70.49180327868852} for 2018
# {'test_accuracy_percent': 60.526315789473685, 'tn_accuracy': 76.19047619047619} for 2017
# {'test_accuracy_percent': 58.620689655172406, 'tn_accuracy': 70.0} for 2016

# MACD CROSSOVER
# {'test_accuracy_percent': 55.24475524475524, 'tn_accuracy': 69.58904109589041} just MACD crossover

# {'test_accuracy_percent': 51.06382978723404, 'tn_accuracy': 71.69811320754717} for 2022
# {'test_accuracy_percent': 41.17647058823529, 'tn_accuracy': 62.06896551724138} for 2021
# {'test_accuracy_percent': 54.929577464788736, 'tn_accuracy': 75.0} for 2020
# {'test_accuracy_percent': 56.71641791044776, 'tn_accuracy': 59.32203389830508} for 2019
# {'test_accuracy_percent': 58.139534883720934, 'tn_accuracy': 67.6923076923077} for 2018
# {'test_accuracy_percent': 61.05263157894737, 'tn_accuracy': 79.24528301886792} for 2017
# {'test_accuracy_percent': 60.43956043956044, 'tn_accuracy': 70.96774193548387} for 2016


In [ ]:
test_acc_dict

# Using KFOLD to train and test the various model

In [ ]:
data_dict = {
    'EURUSD_TRADES': (trade_df.iloc[:, :-1].to_numpy(), trade_df.iloc[:, -1] ),
    # 'EURUSD_TRADES_SS': (StandardScaler().fit_transform(trade_df.iloc[:, :-1].to_numpy()), trade_df.iloc[:, -1] ),
    # 'EURUSD_TRADES_PT': (PowerTransformer().fit_transform(trade_df.iloc[:, :-1].to_numpy()), trade_df.iloc[:, -1] ),
}

In [ ]:
import pandas as pd
import numpy as np
import plotnine as p9
from sklearn.model_selection import *
from sklearn.feature_selection import *
from sklearn.preprocessing import *
from sklearn.pipeline import *

from sklearn.linear_model import *
from sklearn.neighbors import *
from sklearn.model_selection import *
from sklearn.dummy import DummyClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import *
import matplotlib.pyplot as plt
import matplotlib
import plotnine as p9

np.set_printoptions(threshold=np.inf)

test_acc_df_list = []


classifier_dict = {
    "KNeighborsClassifier": KNeighborsClassifier(10),
    # "SVC": SVC(kernel="linear", C=0.025),
    # "SVC": SVC(gamma=2, C=1),
    # "GaussianProcessClassifier": GaussianProcessClassifier(1.0 * RBF(1.0)),
    # "DecisionTreeClassifier": DecisionTreeClassifier(max_depth=5),
    # "MLPClassifier": MLPClassifier(),
    # "AdaBoostClassifier": AdaBoostClassifier(),
    # "GaussianNB": GaussianNB(),
    # "QuadraticDiscriminantAnalysis": QuadraticDiscriminantAnalysis(),
    "RandomForestClassifier": RandomForestClassifier(),
    'DummyClassifier': DummyClassifier(strategy="uniform"),
    'LogisticRegressionCV': LogisticRegressionCV(cv=3, random_state=0, max_iter=1_000),
    
}

for data_set, (input_mat, output_vec) in data_dict.items():
    kf = KFold(n_splits=3, shuffle=True, random_state=1)
    for fold_id, (train_index, test_index) in enumerate(kf.split(input_mat)):
        X_train, X_test = input_mat[train_index], input_mat[test_index]
        y_train, y_test = output_vec[train_index], output_vec[test_index]

        pred_dict = {}
        # iterate over classifiers
        for name, clf in classifier_dict.items():
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            pred_dict[name] = y_pred
        
        for algorithm, y_pred in pred_dict.items():
            tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
            tn_accuracy = tn / (tn + fp)
            
            test_acc_dict = {
                "test_accuracy_percent": accuracy_score(y_test, y_pred) * 100,
                "tn_accuracy": tn_accuracy * 100,
                # "precision_score": precision_score(y_test, y_pred),
                # "f1_score": f1_score(y_test, y_pred),
                "data_set": data_set,
                "fold_id": fold_id,
                "algorithm": algorithm
            }
            test_acc_df_list.append(pd.DataFrame(test_acc_dict, index=[0]))
test_acc_df = pd.concat(test_acc_df_list)


test_acc_df = pd.concat(test_acc_df_list)
gg = p9.ggplot() +\
    p9.geom_point(
        p9.aes(
            x="tn_accuracy",
            y="algorithm",
            color = "factor(fold_id)",
        ),
        data=test_acc_df) +\
    p9.facet_wrap("data_set") +\
    p9.scale_color_manual(values=["red", "blue", "green", "yellow", "pink"], name= "Fold ID")
gg


In [ ]:
y_pred

In [ ]:
mean_test_acc_df = test_acc_df.groupby(["data_set", "algorithm"]).mean().reset_index()


gg = p9.ggplot() +\
    p9.geom_point(
        p9.aes(
            x="tn_accuracy",
            y="algorithm",
            color = "factor(fold_id)",
        ),
        data=mean_test_acc_df) +\
    p9.facet_wrap("data_set") +\
    p9.scale_color_manual(values=["red", "blue", "green"], name= "Fold ID")
gg
